In [193]:
# ! pip install yfinance

In [194]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [195]:
data_path = '/Users/vivianzhao/Desktop/merged_monthly.csv'
tickers = ["PFE"] # company to do hypothesis testing on

In [196]:
# prepare testing stock return
start_date = "2019-12-01"
end_date = datetime.today().strftime('2024-7-31')

# Initialize an empty DataFrame to store all the data
all_data = pd.DataFrame(columns=["Date", "Adj Close", "Close", "High", "Low", "Open", "Volume"])
# Loop through each ticker to fetch data
for ticker in tickers:
    print(f"Fetching data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    data['Ticker'] = ticker  # Add the ticker column
    data.reset_index(inplace=True)  # Reset the index to include the date column
    data.columns = ["Date","Adj Close", "Close", "High", "Low", "Open", "Volume"]
    all_data = pd.concat([data, all_data], ignore_index=True)

testing_data = pd.DataFrame()
testing_data['Daily_Return'] = (all_data['Close'] - all_data['Close'].shift(1)) / all_data['Close'].shift(1)
testing_data['Date'] = all_data['Date']

[*********************100%***********************]  1 of 1 completed

Fetching data for PFE...


In [197]:
testing_data

,Daily_Return,Date
0,NaN,2019-12-02
1,-0.015464,2019-12-03
2,0.001571,2019-12-04
3,0.002875,2019-12-05
4,0.003909,2019-12-06
...,...,...
1167,0.011808,2024-07-24
1168,0.031344,2024-07-25
1169,0.012473,2024-07-26
1170,0.001619,2024-07-29


In [198]:
# prepare macro data
df = pd.read_csv(data_path)
df.head()

,observation_date,Money_Supply_M2,Money_Supply_M1,Interest_Rate,PPI,Real_Dollar_Index,Unemployment_Rate,CPI,GDP
0,2019-12-31,15334.3,4008.4,1.55,199.0,107.1923,3.6,258.630,21933.217
1,2020-01-31,15401.3,3977.6,1.59,199.3,106.5735,3.6,258.906,21727.657
2,2020-02-29,15453.8,3979.6,1.58,196.7,107.8561,3.5,259.246,21727.657
3,2020-03-31,15980.6,4260.9,0.08,193.1,111.8032,4.4,258.150,21727.657
4,2020-04-30,16999.0,4788.8,0.05,185.5,113.4208,14.8,256.126,19935.444


In [199]:
# standardize data
from sklearn.preprocessing import StandardScaler

independent_vars = df[['Money_Supply_M2', 'Money_Supply_M1',
       'Interest_Rate', 'PPI', 'Real_Dollar_Index', 'Unemployment_Rate', 'CPI',
       'GDP']]

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
standardized_vars = scaler.fit_transform(independent_vars)

# Create a DataFrame with standardized variables
standardized_df = pd.DataFrame(standardized_vars, columns=independent_vars.columns)


standardized_df['observation_date'] = df['observation_date']

df = standardized_df

In [200]:
# Convert date column to datetime
df["observation_date"] = pd.to_datetime(df["observation_date"])

In [201]:
# Extract year and month
df["year"] = df["observation_date"].dt.year
df["month"] = df["observation_date"].dt.month

# Identify quarters
df["quarter"] = df["observation_date"].dt.to_period("Q")

df.head()

,Money_Supply_M2,Money_Supply_M1,Interest_Rate,PPI,Real_Dollar_Index,Unemployment_Rate,CPI,GDP,observation_date,year,month,quarter
0,-2.862721,-3.119522,-0.308750,-1.381540,-0.983079,-0.582715,-1.318769,-1.184124,2019-12-31,2019,12,2019Q4
1,-2.822886,-3.126621,-0.291338,-1.370720,-1.119489,-0.582715,-1.305061,-1.259381,2020-01-31,2020,1,2020Q1
2,-2.791672,-3.126160,-0.295691,-1.464497,-0.836750,-0.624285,-1.288174,-1.259381,2020-02-29,2020,2,2020Q1
3,-2.478460,-3.061330,-0.948637,-1.594341,0.033356,-0.250159,-1.342609,-1.259381,2020-03-31,2020,3,2020Q1
4,-1.872965,-2.939669,-0.961696,-1.868457,0.389943,4.073068,-1.443134,-1.915522,2020-04-30,2020,4,2020Q2


In [202]:
df["month_position"] = df["month"] % 3  # 0 = first month, 1 = second month, 2 = third month

# Create separate DataFrames for each month's position in the quarter
df_first_month = df[df["month_position"] == 1].copy()
df_second_month = df[df["month_position"] == 2].copy()
df_third_month = df[df["month_position"] == 0].copy()  

In [203]:
# Merge by quarter 
df_quarterly = df_first_month[["quarter", "Money_Supply_M2", "Money_Supply_M1", "Interest_Rate", "PPI",
                               "Real_Dollar_Index", "Unemployment_Rate", "CPI"]].rename(
    columns=lambda x: x + "_M1" if x != "quarter" else x)

df_quarterly = df_quarterly.merge(
    df_second_month[["quarter", "Money_Supply_M2", "Money_Supply_M1", "Interest_Rate", "PPI",
                     "Real_Dollar_Index", "Unemployment_Rate", "CPI"]].rename(
        columns=lambda x: x + "_M2" if x != "quarter" else x),
    on="quarter", how="inner"
)

df_quarterly = df_quarterly.merge(
    df_third_month[["quarter", "Money_Supply_M2", "Money_Supply_M1", "Interest_Rate", "PPI",
                    "Real_Dollar_Index", "Unemployment_Rate", "CPI", "GDP"]].rename(
        columns=lambda x: x + "_M3" if x not in ["quarter", "GDP"] else x),
    on="quarter", how="inner"
)

In [204]:
df_quarterly.head()

,quarter,Money_Supply_M2_M1,Money_Supply_M1_M1,Interest_Rate_M1,PPI_M1,Real_Dollar_Index_M1,Unemployment_Rate_M1,CPI_M1,Money_Supply_M2_M2,Money_Supply_M1_M2,...,Unemployment_Rate_M2,CPI_M2,Money_Supply_M2_M3,Money_Supply_M1_M3,Interest_Rate_M3,PPI_M3,Real_Dollar_Index_M3,Unemployment_Rate_M3,CPI_M3,GDP
0,2020Q1,-2.822886,-3.126621,-0.291338,-1.370720,-1.119489,-0.582715,-1.305061,-2.791672,-3.126160,...,-0.624285,-1.288174,-2.478460,-3.061330,-0.948637,-1.594341,0.033356,-0.250159,-1.342609,-1.259381
1,2020Q2,-1.872965,-2.939669,-0.961696,-1.868457,0.389943,4.073068,-1.443134,-1.356058,-0.299318,...,3.407956,-1.456941,-1.181794,-0.223587,-0.948637,-1.662870,-0.345319,2.493427,-1.399527,-1.915522
2,2020Q3,-1.092670,-0.177403,-0.939931,-1.597948,-0.476702,2.160871,-1.329795,-1.050575,-0.148756,...,1.412620,-1.282214,-0.917038,-0.086139,-0.944284,-1.507778,-0.853349,1.163203,-1.253159,-1.275163
3,2020Q4,-0.827082,-0.041153,-0.944284,-1.471710,-0.972586,0.789078,-1.238358,-0.699371,0.015818,...,0.705939,-1.206274,-0.617917,0.059629,-0.944284,-1.327439,-1.700133,0.705939,-1.151144,-1.134499
4,2021Q1,-0.484321,0.119573,-0.952990,-1.172347,-1.864031,0.581231,-1.125665,-0.343887,0.179309,...,0.498092,-1.072770,-0.201907,0.242318,-0.957343,-0.804454,-1.382498,0.456522,-1.006862,-0.919219


In [205]:
# Calculate Quarterly Returns
testing_data["Date"] = pd.to_datetime(testing_data["Date"])
testing_data["quarter"] = testing_data["Date"].dt.to_period("Q")

quarterly_returns = (
    testing_data.groupby("quarter")["Daily_Return"]
    .apply(lambda x: (1 + x).prod() - 1)
    .reset_index()
    .rename(columns={"Daily_Return": "Quarterly_Return"})
)
quarterly_returns.head()

,quarter,Quarterly_Return
0,2019Q4,0.010310
1,2020Q1,-0.149564
2,2020Q2,0.006293
3,2020Q3,0.135074
4,2020Q4,0.063687


In [206]:
# Merge quarterly returns with independent variables on the quarter
merged_data = pd.merge(quarterly_returns, df_quarterly, on="quarter", how="inner")
merged_data.head()

,quarter,Quarterly_Return,Money_Supply_M2_M1,Money_Supply_M1_M1,Interest_Rate_M1,PPI_M1,Real_Dollar_Index_M1,Unemployment_Rate_M1,CPI_M1,Money_Supply_M2_M2,...,Unemployment_Rate_M2,CPI_M2,Money_Supply_M2_M3,Money_Supply_M1_M3,Interest_Rate_M3,PPI_M3,Real_Dollar_Index_M3,Unemployment_Rate_M3,CPI_M3,GDP
0,2020Q1,-0.149564,-2.822886,-3.126621,-0.291338,-1.370720,-1.119489,-0.582715,-1.305061,-2.791672,...,-0.624285,-1.288174,-2.478460,-3.061330,-0.948637,-1.594341,0.033356,-0.250159,-1.342609,-1.259381
1,2020Q2,0.006293,-1.872965,-2.939669,-0.961696,-1.868457,0.389943,4.073068,-1.443134,-1.356058,...,3.407956,-1.456941,-1.181794,-0.223587,-0.948637,-1.662870,-0.345319,2.493427,-1.399527,-1.915522
2,2020Q3,0.135074,-1.092670,-0.177403,-0.939931,-1.597948,-0.476702,2.160871,-1.329795,-1.050575,...,1.412620,-1.282214,-0.917038,-0.086139,-0.944284,-1.507778,-0.853349,1.163203,-1.253159,-1.275163
3,2020Q4,0.063687,-0.827082,-0.041153,-0.944284,-1.471710,-0.972586,0.789078,-1.238358,-0.699371,...,0.705939,-1.206274,-0.617917,0.059629,-0.944284,-1.327439,-1.700133,0.705939,-1.151144,-1.134499
4,2021Q1,-0.002538,-0.484321,0.119573,-0.952990,-1.172347,-1.864031,0.581231,-1.125665,-0.343887,...,0.498092,-1.072770,-0.201907,0.242318,-0.957343,-0.804454,-1.382498,0.456522,-1.006862,-0.919219


In [207]:
import pandas as pd
import statsmodels.api as sm

# Define the dependent variable and exclude non-numeric columns
dependent_var = "Quarterly_Return"
exclude_columns = ["Quarter", "observation_date", dependent_var]  # Add other non-numeric columns here

# Filter numeric independent variables
independent_vars = ['Money_Supply_M2', 'Money_Supply_M1', 'Interest_Rate', 'PPI',
       'Real_Dollar_Index', 'CPI', 'Unemployment_Rate']


# Initialize a dictionary to store results
best_month_results = {}

# Loop through each independent variable
for var in independent_vars:
    months = ["M1", "M2", "M3"]
    results = {}

    # Loop through each month
    for month in months:
        # Select the variable for the specific month
        X = merged_data[[f"{var}_{month}"]]
        X = sm.add_constant(X)
        y = merged_data[dependent_var]
        
        # Fit the regression model
        model = sm.OLS(y, X).fit()
        
        # Store p-value and R^2
        results[month] = {
            "p_value": model.pvalues.get(f"{var}_{month}", float("inf")),
            "R2": model.rsquared,
            "Summary": model.summary()
        }
    
    # Identify the best month (based on lowest p-value or highest R^2)
    best_month = min(results, key=lambda m: results[m]["p_value"])
    best_result = results[best_month]
    
    # Store the best month's result for this variable
    best_month_results[var] = {
        "Best Month": best_month,
        "P-Value": best_result["p_value"],
        "R2": best_result["R2"]
    }

# Convert the results to a DataFrame for better readability
best_month_df = pd.DataFrame.from_dict(best_month_results, orient="index")

# Display the results
print(best_month_df)


                  Best Month   P-Value        R2
Money_Supply_M2           M3  0.597021  0.017861
Money_Supply_M1           M3  0.226095  0.090136
Interest_Rate             M1  0.056076  0.209588
PPI                       M1  0.367476  0.051017
Real_Dollar_Index         M3  0.091875  0.167331
CPI                       M1  0.188111  0.105664
Unemployment_Rate         M1  0.298910  0.067202


/Users/vivianzhao/anaconda3/envs/dsc80/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/vivianzhao/anaconda3/envs/dsc80/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/vivianzhao/anaconda3/envs/dsc80/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/vivianzhao/anaconda3/envs/dsc80/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/vivianzhao/anaconda3/envs/dsc